In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install langchain_community langchain_openai faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

tools = [weather, multiply]
llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


ensolaradoO clima em Recife está ensolarado.

> Finished chain.


{'input': 'Qual o clima em Recife?',
 'output': 'O clima em Recife está ensolarado.'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [10]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()


··········


In [11]:
!pip install langchain_community langchain_openai faiss-cpu


In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [13]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def create_text_file(filename: str, content: str) -> str:
    """Cria um arquivo de texto com o nome e conteúdo fornecidos"""
    with open(filename, "w") as file:
        file.write(content)
    return f"Arquivo {filename} criado com sucesso."

In [14]:
# Adicionando a nova ferramenta à lista
tools = [weather, multiply, create_text_file]
llm_with_tools = llm.bind_tools(tools)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [16]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [19]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Testando a nova função
response = agent_executor.invoke({"input": "Crie um arquivo chamado 'arquivo.txt' com o conteúdo 'Este é o arquivo gerado para teste do agente.'"})
print(response)




> Entering new AgentExecutor chain...

Invoking: `create_text_file` with `{'filename': 'arquivo.txt', 'content': 'Este é o arquivo gerado para teste do agente.'}`


Arquivo arquivo.txt criado com sucesso.O arquivo 'arquivo.txt' foi criado com sucesso, contendo o texto: "Este é o arquivo gerado para teste do agente."

> Finished chain.
{'input': "Crie um arquivo chamado 'arquivo.txt' com o conteúdo 'Este é o arquivo gerado para teste do agente.'", 'output': 'O arquivo \'arquivo.txt\' foi criado com sucesso, contendo o texto: "Este é o arquivo gerado para teste do agente."'}


### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [23]:
@tool
def count_letter(word: str, letter: str) -> int:
    """Conta quantas vezes uma letra aparece em uma palavra"""
    return word.lower().count(letter.lower())

# Adicionando a nova ferramenta à lista
tools = [weather, multiply, create_text_file, count_letter]
llm_with_tools = llm.bind_tools(tools)



In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

# Criando o prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [27]:
# Configurando o agente
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

# Executando o agente
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [28]:
# Testando o agente com a nova função
response = agent_executor.invoke({"input": "Conte quantas vezes a letra 'p' aparece na palavra 'paralelepipedo'."})
print(response)



> Entering new AgentExecutor chain...

Invoking: `count_letter` with `{'word': 'paralelepipedo', 'letter': 'p'}`


3A letra 'p' aparece 3 vezes na palavra 'paralelepipedo'.

> Finished chain.
{'input': "Conte quantas vezes a letra 'p' aparece na palavra 'paralelepipedo'.", 'output': "A letra 'p' aparece 3 vezes na palavra 'paralelepipedo'."}
